In [1]:
import os
import pickle5 as pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import warnings
warnings.filterwarnings('ignore')

# Import Path,Vocabulary, utility, evaluator and datahandler module
from config import Path
from dictionary import Vocabulary
from utils import Utils
from evaluate import Evaluator
from data import DataHandler
from coaha import COAHA


import random
import numpy as np
import copy
from tqdm import tqdm

#set seed for reproducibility
utils = Utils()
utils.set_seed(1)

/home/nasibullah/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/nasibullah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and g

In [2]:
#Import configuration and model 
from config import ConfigTHVC
from models.THVC.model import THVC

cfg = ConfigTHVC()

# Change of Hyperparameters
cfg.dataset = 'msvd'
cfg.vocabulary_min_count = 5
cfg.create_entity = False


#creation of path object
path = Path(cfg,os.getcwd())
#Vocabulary object 
voc = Vocabulary(cfg)
#If vocabulary is already saved or downloaded the saved file
voc.load() #comment this if using vocabulary for the first time or with no saved file
min_count = cfg.vocabulary_min_count #remove all words below count min_count
voc.trim(min_count=min_count)
print('Vocabulary Size : ',voc.num_words)

Saved/msvd_word2index_dic.p
keep_words 3981 / 12596 = 0.3161
Vocabulary Size :  3984


# Setting Up DataLoaders, Model and Evaluators (Standard and COAHA)

In [3]:
# Datasets and dataloaders
data_handler = DataHandler(cfg,path,voc)
train_dset,val_dset,test_dset = data_handler.getDatasets()
train_loader,val_loader,test_loader = data_handler.getDataloader(train_dset,val_dset,test_dset)
if cfg.opt_auxiliary_heads:
    cfg.update_head_info(data_handler.auxhead_data)
    
#Model object
model = THVC(voc,cfg,path)
#Standard Evaluator
#Evaluator object on test data
test_evaluator_greedy = Evaluator(model,test_loader,path,cfg,data_handler.test_dict)
#test_evaluator_beam = Evaluator(model,test_loader,path,cfg,data_handler.test_dict,decoding_type='beam')

#COAHA evaluator
coaha_test = COAHA(cfg,data_handler.test_dict)

  0%|          | 0/670 [00:00<?, ?it/s]

Setting up COAHA object...


100%|██████████| 670/670 [03:15<00:00,  3.43it/s]


# Training Loop

In [ ]:
#Training Loop
cfg.encoder_lr = 1e-4
cfg.decoder_lr = 1e-4
cfg.teacher_forcing_ratio = 1.0

model.update_hyperparameters(cfg)
coaha = []

#lr_scheduler = StepLR(model.dec_optimizer,300,gamma=0.1,verbose=False)
# lr_scheduler = ReduceLROnPlateau(model.dec_optimizer, mode='min', factor=cfg.lr_decay_gamma,
#                                      patience=cfg.lr_decay_patience, verbose=True)
for e in range(1,901):
    loss_train,ac_loss = model.train_epoch(train_loader,utils)
    #lr_scheduler.step()
    if e%25 == 0 :
        print('Epoch -- >',e,'Loss -->',loss_train,'  AC loss --->',ac_loss)
        print('greedy :',test_evaluator_greedy.evaluate(utils,model,e,loss_train))
        #coaha_test.evaluate(test_evaluator_greedy.prediction_dict)
        #print('COAHA SCORE: ',coaha_test.coaha_total)
        #coaha.append(coaha_test.coaha_total)


# Load Pre-trained Model

In [4]:
model.load_state_dict(torch.load('saved_model.pt'))
model.eval()
test_evaluator_greedy.evaluate(utils,model,900,999)

{'testlen': 4048, 'reflen': 4045, 'guess': [4048, 3378, 2708, 2038], 'correct': [3378, 2131, 1287, 655]}
ratio: 1.0007416563656364


{'Bleu_1': 0.834486166007699,
 'Bleu_2': 0.7255568939261929,
 'Bleu_3': 0.6301214701404569,
 'Bleu_4': 0.532509657260048,
 'METEOR': 0.36549265103712764,
 'ROUGE_L': 0.7398569346535218,
 'CIDEr': 0.998611514550787}

In [5]:
coaha_test.evaluate(test_evaluator_greedy.prediction_dict)
coaha_test.coaha_total

100%|██████████| 670/670 [00:43<00:00, 15.45it/s]


0.0701640281264076